# **Homework 7 - Bert (Question Answering)**

If you have any questions, feel free to email us at mlta-2022-spring@googlegroups.com



Slide:    [Link](https://docs.google.com/presentation/d/1H5ZONrb2LMOCixLY7D5_5-7LkIaXO6AGEaV2mRdTOMY/edit?usp=sharing)　Kaggle: [Link](https://www.kaggle.com/c/ml2022spring-hw7)　Data: [Link](https://drive.google.com/uc?id=1AVgZvy3VFeg0fX-6WQJMHPVrx3A-M1kb)




## Task description
- Chinese Extractive Question Answering
  - Input: Paragraph + Question
  - Output: Answer

- Objective: Learn how to fine tune a pretrained model on downstream task using transformers

- Todo
    - Fine tune a pretrained chinese BERT model
    - Change hyperparameters (e.g. doc_stride)
    - Apply linear learning rate decay
    - Try other pretrained models
    - Improve preprocessing
    - Improve postprocessing
- Training tips
    - Automatic mixed precision
    - Gradient accumulation
    - Ensemble

- Estimated training time (tesla t4 with automatic mixed precision enabled)
    - Simple: 8mins
    - Medium: 8mins
    - Strong: 25mins
    - Boss: 2.5hrs
  

## Download Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Download link 1
!gdown --id '1AVgZvy3VFeg0fX-6WQJMHPVrx3A-M1kb' --output hw7_data.zip

# Download Link 2 (if the above link fails) 
# !gdown --id '1qwjbRjq481lHsnTrrF4OjKQnxzgoLEFR' --output hw7_data.zip

# Download Link 3 (if the above link fails) 
# !gdown --id '1QXuWjNRZH6DscSd6QcRER0cnxmpZvijn' --output hw7_data.zip

!unzip -o hw7_data.zip

# For this HW, K80 < P4 < T4 < P100 <= T4(fp16) < V100
!nvidia-smi

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1AVgZvy3VFeg0fX-6WQJMHPVrx3A-M1kb
To: /content/hw7_data.zip
100% 9.57M/9.57M [00:00<00:00, 194MB/s]
Archive:  hw7_data.zip
  inflating: hw7_dev.json            
  inflating: hw7_test.json           
  inflating: hw7_train.json          
Sun Apr 17 13:47:29 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      

## Install transformers

Documentation for the toolkit:　https://huggingface.co/transformers/

In [ ]:
# You are allowed to change version of transformers or use other toolkits
!pip install transformers==4.5.0

     |████████████████████████████████| 2.1 MB 14.3 MB/s 
     |████████████████████████████████| 895 kB 73.4 MB/s 
     |████████████████████████████████| 3.3 MB 64.0 MB/s 


## Import Packages

In [ ]:
import json
import numpy as np
import random
import torch
from torch.utils.data import DataLoader, Dataset 
from transformers import AdamW, BertForQuestionAnswering, BertTokenizerFast, get_linear_schedule_with_warmup

from tqdm.auto import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"

# Fix random seed for reproducibility
def same_seeds(seed):
	  torch.manual_seed(seed)
	  if torch.cuda.is_available():
		    torch.cuda.manual_seed(seed)
		    torch.cuda.manual_seed_all(seed)
	  np.random.seed(seed)
	  random.seed(seed)
	  torch.backends.cudnn.benchmark = False
	  torch.backends.cudnn.deterministic = True
same_seeds(0)

In [ ]:
# Change "fp16_training" to True to support automatic mixed precision training (fp16)	
fp16_training = True

if fp16_training:
    !pip install accelerate==0.2.0
    from accelerate import Accelerator
    accelerator = Accelerator(fp16=True)
    device = accelerator.device

# Documentation for the toolkit:  https://huggingface.co/docs/accelerate/

     |████████████████████████████████| 47 kB 4.3 MB/s 


## Load Model and Tokenizer




 

In [ ]:
model = BertForQuestionAnswering.from_pretrained("/content/drive/MyDrive/ML/HW7").to(device)
tokenizer = BertTokenizerFast.from_pretrained("hfl/chinese-macbert-large")

# You can safely ignore the warning message (it pops up because new prediction heads for QA are initialized randomly)

Downloading:   0%|          | 0.00/110k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/269k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/19.0 [00:00<?, ?B/s]

## Read Data

- Training set: 31690 QA pairs
- Dev set: 4131  QA pairs
- Test set: 4957  QA pairs

- {train/dev/test}_questions:	
  - List of dicts with the following keys:
   - id (int)
   - paragraph_id (int)
   - question_text (string)
   - answer_text (string)
   - answer_start (int)
   - answer_end (int)
- {train/dev/test}_paragraphs: 
  - List of strings
  - paragraph_ids in questions correspond to indexs in paragraphs
  - A paragraph may be used by several questions 

In [ ]:
def read_data(file):
    with open(file, 'r', encoding="utf-8") as reader:
        data = json.load(reader)
    return data["questions"], data["paragraphs"]

train_questions, train_paragraphs = read_data("hw7_train.json")
dev_questions, dev_paragraphs = read_data("hw7_dev.json")
test_questions, test_paragraphs = read_data("hw7_test.json")

## Tokenize Data

In [ ]:
# Tokenize questions and paragraphs separately
# 「add_special_tokens」 is set to False since special tokens will be added when tokenized questions and paragraphs are combined in datset __getitem__ 

train_questions_tokenized = tokenizer([train_question["question_text"] for train_question in train_questions], add_special_tokens=False)
dev_questions_tokenized = tokenizer([dev_question["question_text"] for dev_question in dev_questions], add_special_tokens=False)
test_questions_tokenized = tokenizer([test_question["question_text"] for test_question in test_questions], add_special_tokens=False) 

train_paragraphs_tokenized = tokenizer(train_paragraphs, add_special_tokens=False)
dev_paragraphs_tokenized = tokenizer(dev_paragraphs, add_special_tokens=False)
test_paragraphs_tokenized = tokenizer(test_paragraphs, add_special_tokens=False)

# You can safely ignore the warning message as tokenized sequences will be futher processed in datset __getitem__ before passing to model

## Dataset and Dataloader

In [ ]:
class QA_Dataset(Dataset):
    def __init__(self, split, questions, tokenized_questions, tokenized_paragraphs):
        self.split = split
        self.questions = questions
        self.tokenized_questions = tokenized_questions
        self.tokenized_paragraphs = tokenized_paragraphs
        self.max_question_len = 40
        self.max_paragraph_len = 350
        
        ##### TODO: Change value of doc_stride #####
        self.doc_stride = 300

        # Input sequence length = [CLS] + question + [SEP] + paragraph + [SEP]
        self.max_seq_len = 1 + self.max_question_len + 1 + self.max_paragraph_len + 1

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        question = self.questions[idx]
        tokenized_question = self.tokenized_questions[idx]
        tokenized_paragraph = self.tokenized_paragraphs[question["paragraph_id"]]

        ##### TODO: Preprocessing #####
        # Hint: How to prevent model from learning something it should not learn

        if self.split == "train":
            # Convert answer's start/end positions in paragraph_text to start/end positions in tokenized_paragraph  
            answer_start_token = tokenized_paragraph.char_to_token(question["answer_start"])
            answer_end_token = tokenized_paragraph.char_to_token(question["answer_end"])

            # A single window is obtained by slicing the portion of paragraph containing the answer
            # mid is not always the answer
            mid = int((answer_start_token + answer_end_token) // (2 + random.uniform(-1,1)))
            paragraph_start = max(0, min(mid - self.max_paragraph_len // 2, len(tokenized_paragraph) - self.max_paragraph_len))
            paragraph_end = paragraph_start + self.max_paragraph_len
            
            # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
            input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102] 
            input_ids_paragraph = tokenized_paragraph.ids[paragraph_start : paragraph_end] + [102]		
            
            # Convert answer's start/end positions in tokenized_paragraph to start/end positions in the window  
            answer_start_token += len(input_ids_question) - paragraph_start
            answer_end_token += len(input_ids_question) - paragraph_start
            
            # Pad sequence and obtain inputs to model 
            input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)
            return torch.tensor(input_ids), torch.tensor(token_type_ids), torch.tensor(attention_mask), answer_start_token, answer_end_token

        # Validation/Testing
        else:
            input_ids_list, token_type_ids_list, attention_mask_list = [], [], []
            
            # Paragraph is split into several windows, each with start positions separated by step "doc_stride"
            for i in range(0, len(tokenized_paragraph), self.doc_stride):
                
                # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
                input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102]
                input_ids_paragraph = tokenized_paragraph.ids[i : i + self.max_paragraph_len] + [102]
                
                # Pad sequence and obtain inputs to model
                input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)
                
                input_ids_list.append(input_ids)
                token_type_ids_list.append(token_type_ids)
                attention_mask_list.append(attention_mask)
            
            return torch.tensor(input_ids_list), torch.tensor(token_type_ids_list), torch.tensor(attention_mask_list)

    def padding(self, input_ids_question, input_ids_paragraph):
        # Pad zeros if sequence length is shorter than max_seq_len
        padding_len = self.max_seq_len - len(input_ids_question) - len(input_ids_paragraph)
        # Indices of input sequence tokens in the vocabulary
        input_ids = input_ids_question + input_ids_paragraph + [0] * padding_len
        # Segment token indices to indicate first and second portions of the inputs. Indices are selected in [0, 1]
        token_type_ids = [0] * len(input_ids_question) + [1] * len(input_ids_paragraph) + [0] * padding_len
        # Mask to avoid performing attention on padding token indices. Mask values selected in [0, 1]
        attention_mask = [1] * (len(input_ids_question) + len(input_ids_paragraph)) + [0] * padding_len
        
        return input_ids, token_type_ids, attention_mask

train_set = QA_Dataset("train", train_questions, train_questions_tokenized, train_paragraphs_tokenized)
dev_set = QA_Dataset("dev", dev_questions, dev_questions_tokenized, dev_paragraphs_tokenized)
test_set = QA_Dataset("test", test_questions, test_questions_tokenized, test_paragraphs_tokenized)

train_batch_size = 4

# Note: Do NOT change batch size of dev_loader / test_loader !
# Although batch size=1, it is actually a batch consisting of several windows from the same QA pair
train_loader = DataLoader(train_set, batch_size=train_batch_size, shuffle=True, pin_memory=True)
dev_loader = DataLoader(dev_set, batch_size=1, shuffle=False, pin_memory=True)
test_loader = DataLoader(test_set, batch_size=1, shuffle=False, pin_memory=True)

## Function for Evaluation

In [ ]:
def evaluate(data, output):
    ##### TODO: Postprocessing #####
    # There is a bug and room for improvement in postprocessing 
    # Hint: Open your prediction file to see what is wrong 
    
    answer = ''
    max_prob = float('-inf')
    num_of_windows = data[0].shape[1]

    for k in range(num_of_windows):
        # Obtain answer by choosing the most probable start position / end position
        mask = data[1][0][k].bool() & data[2][0][k].bool() # attention mask
        mask = mask.to(device)
        masked_output_start = torch.masked_select(output.start_logits[k], mask)[:-1].to(device)
        start_prob, start_index = torch.max(masked_output_start, dim=0)
        masked_output_end = torch.masked_select(output.end_logits[k], mask)[start_index:-1].to(device)
        end_prob, end_index = torch.max(masked_output_end, dim=0)
        end_index += start_index 
        # Probability of answer is calculated as sum of start_prob and end_prob
        prob = start_prob + end_prob
        masked_data = torch.masked_select(data[0][0][k].to(device), mask)[:-1].to(device)
        # Replace answer if calculated probability is larger than previous windows
        print(start_index,end_index)
        if (prob > max_prob) and (end_index >= start_index) and (end_index - start_index <= 30):
            max_prob = prob
            # Convert tokens to chars (e.g. [1920, 7032] --> "大 金")
            #answer = tokenizer.decode(data[0][0][k][start_index : end_index + 1])
            answer = tokenizer.decode(masked_data[start_index : end_index + 1])

    
    # Remove spaces in answer (e.g. "大 金" --> "大金")
    answer = answer.replace(' ','')
    print("prediction",answer)
    return answer

## Training

In [ ]:
num_epoch = 3
validation = True
logging_step = 100
learning_rate = 1e-5
doc_stride = 300
optimizer = AdamW(model.parameters(), lr=learning_rate)
total_steps = len(train_loader) * num_epoch
scheduler = get_linear_schedule_with_warmup(optimizer,num_warmup_steps=0,num_training_steps=total_steps)
if fp16_training:
    model, optimizer, train_loader = accelerator.prepare(model, optimizer, train_loader) 

model.train()

print("Start Training ...")

for epoch in range(num_epoch):
    step = 1
    train_loss = train_acc = 0
    
    for data in tqdm(train_loader):	
        # Load all data into GPU
        data = [i.to(device) for i in data]
        
        # Model inputs: input_ids, token_type_ids, attention_mask, start_positions, end_positions (Note: only "input_ids" is mandatory)
        # Model outputs: start_logits, end_logits, loss (return when start_positions/end_positions are provided)  
        output = model(input_ids=data[0], token_type_ids=data[1], attention_mask=data[2], start_positions=data[3], end_positions=data[4])

        # Choose the most probable start position / end position
        start_index = torch.argmax(output.start_logits, dim=1)
        end_index = torch.argmax(output.end_logits, dim=1)
        
        # Prediction is correct only if both start_index and end_index are correct
        train_acc += ((start_index == data[3]) & (end_index == data[4])).float().mean()
        train_loss += output.loss
        
        if fp16_training:
            accelerator.backward(output.loss)
        else:
            output.loss.backward()
        
        optimizer.step()
        optimizer.zero_grad()
        step += 1
        ##### TODO: Apply linear learning rate decay #####
        
        
        
        # Print training loss and accuracy over past logging step
        if step % logging_step == 0:
            print(f"Epoch {epoch + 1} | Step {step} | loss = {train_loss.item() / logging_step:.3f}, acc = {train_acc / logging_step:.3f}")
            train_loss = train_acc = 0
    scheduler.step()
    if validation:
        print("Evaluating Dev Set ...")
        model.eval()
        with torch.no_grad():
            dev_acc = 0
            for i, data in enumerate(tqdm(dev_loader)):
                output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
                # prediction is correct only if answer text exactly matches
                dev_acc += (evaluate(data, output) == dev_questions[i]["answer_text"])
            print(f"Validation | Epoch {epoch + 1} | acc = {dev_acc / len(dev_loader):.3f}")
        model.train()

# Save a model and its configuration file to the directory 「saved_model」 
# i.e. there are two files under the direcory 「saved_model」: 「pytorch_model.bin」 and 「config.json」
# Saved model can be re-loaded using 「model = BertForQuestionAnswering.from_pretrained("saved_model")」
print("Saving Model ...")
model_save_dir = "/content/drive/MyDrive/ML/HW7" 
model.save_pretrained(model_save_dir)

Start Training ...


  0%|          | 0/7923 [00:00<?, ?it/s]

Epoch 1 | Step 100 | loss = 0.066, acc = 0.957
Epoch 1 | Step 200 | loss = 0.103, acc = 0.957
Epoch 1 | Step 300 | loss = 0.092, acc = 0.970
Epoch 1 | Step 400 | loss = 0.113, acc = 0.970
Epoch 1 | Step 500 | loss = 0.121, acc = 0.952
Epoch 1 | Step 600 | loss = 0.139, acc = 0.952
Epoch 1 | Step 700 | loss = 0.062, acc = 0.970
Epoch 1 | Step 800 | loss = 0.114, acc = 0.952
Epoch 1 | Step 900 | loss = 0.091, acc = 0.952
Epoch 1 | Step 1000 | loss = 0.107, acc = 0.960
Epoch 1 | Step 1100 | loss = 0.101, acc = 0.967
Epoch 1 | Step 1200 | loss = 0.057, acc = 0.967
Epoch 1 | Step 1300 | loss = 0.052, acc = 0.967
Epoch 1 | Step 1400 | loss = 0.053, acc = 0.980
Epoch 1 | Step 1500 | loss = 0.089, acc = 0.962
Epoch 1 | Step 1600 | loss = 0.088, acc = 0.950
Epoch 1 | Step 1700 | loss = 0.046, acc = 0.972
Epoch 1 | Step 1800 | loss = 0.056, acc = 0.972
Epoch 1 | Step 1900 | loss = 0.057, acc = 0.972
Epoch 1 | Step 2000 | loss = 0.085, acc = 0.960
Epoch 1 | Step 2100 | loss = 0.056, acc = 0.970
E

  0%|          | 0/4131 [00:00<?, ?it/s]

串流輸出內容已截斷至最後 5000 行。
tensor(36, device='cuda:0') tensor(40, device='cuda:0')
prediction 《粵大記》
tensor(207, device='cuda:0') tensor(214, device='cuda:0')
tensor(14, device='cuda:0') tensor(17, device='cuda:0')
prediction 歐內斯特·拉塞福
tensor(314, device='cuda:0') tensor(315, device='cuda:0')
tensor(14, device='cuda:0') tensor(15, device='cuda:0')
prediction 1927年
tensor(23, device='cuda:0') tensor(24, device='cuda:0')
tensor(26, device='cuda:0') tensor(26, device='cuda:0')
prediction 1992年
tensor(200, device='cuda:0') tensor(201, device='cuda:0')
tensor(29, device='cuda:0') tensor(30, device='cuda:0')
prediction 倫敦
tensor(27, device='cuda:0') tensor(28, device='cuda:0')
tensor(82, device='cuda:0') tensor(82, device='cuda:0')
prediction 1969年
tensor(0, device='cuda:0') tensor(20, device='cuda:0')
tensor(21, device='cuda:0') tensor(44, device='cuda:0')
prediction 1920年台灣地方改制，日本以此地位於半屏山之東
tensor(242, device='cuda:0') tensor(246, device='cuda:0')
tensor(15, device='cuda:0') tensor(15, device='cud

...

串流輸出內容已截斷至最後 5000 行。
tensor(36, device='cuda:0') tensor(40, device='cuda:0')
prediction 《粵大記》
tensor(207, device='cuda:0') tensor(214, device='cuda:0')
tensor(14, device='cuda:0') tensor(17, device='cuda:0')
prediction 歐內斯特·拉塞福
tensor(314, device='cuda:0') tensor(315, device='cuda:0')
tensor(14, device='cuda:0') tensor(15, device='cuda:0')
prediction 1927年
tensor(23, device='cuda:0') tensor(24, device='cuda:0')
tensor(107, device='cuda:0') tensor(107, device='cuda:0')
prediction 1992年
tensor(200, device='cuda:0') tensor(201, device='cuda:0')
tensor(29, device='cuda:0') tensor(31, device='cuda:0')
prediction 倫敦
tensor(27, device='cuda:0') tensor(28, device='cuda:0')
tensor(91, device='cuda:0') tensor(91, device='cuda:0')
prediction 1969年
tensor(14, device='cuda:0') tensor(20, device='cuda:0')
tensor(5, device='cuda:0') tensor(44, device='cuda:0')
prediction 位於半屏山之東
tensor(242, device='cuda:0') tensor(246, device='cuda:0')
tensor(35, device='cuda:0') tensor(35, device='cuda:0')
predictio

  0%|          | 0/7923 [00:00<?, ?it/s]

Epoch 3 | Step 100 | loss = 0.047, acc = 0.972
Epoch 3 | Step 200 | loss = 0.085, acc = 0.965
Epoch 3 | Step 300 | loss = 0.071, acc = 0.977
Epoch 3 | Step 400 | loss = 0.051, acc = 0.965
Epoch 3 | Step 500 | loss = 0.071, acc = 0.967
Epoch 3 | Step 600 | loss = 0.056, acc = 0.965
Epoch 3 | Step 700 | loss = 0.052, acc = 0.975
Epoch 3 | Step 800 | loss = 0.067, acc = 0.962
Epoch 3 | Step 900 | loss = 0.100, acc = 0.972
Epoch 3 | Step 1000 | loss = 0.077, acc = 0.952
Epoch 3 | Step 1100 | loss = 0.100, acc = 0.942
Epoch 3 | Step 1200 | loss = 0.053, acc = 0.977
Epoch 3 | Step 1300 | loss = 0.039, acc = 0.980
Epoch 3 | Step 1400 | loss = 0.069, acc = 0.972
Epoch 3 | Step 1500 | loss = 0.032, acc = 0.987
Epoch 3 | Step 1600 | loss = 0.062, acc = 0.975
Epoch 3 | Step 1700 | loss = 0.080, acc = 0.960
Epoch 3 | Step 1800 | loss = 0.078, acc = 0.965
Epoch 3 | Step 1900 | loss = 0.060, acc = 0.967
Epoch 3 | Step 2000 | loss = 0.040, acc = 0.980
Epoch 3 | Step 2100 | loss = 0.071, acc = 0.965
E

  0%|          | 0/4131 [00:00<?, ?it/s]

串流輸出內容已截斷至最後 5000 行。
prediction 《粵大記》
tensor(207, device='cuda:0') tensor(214, device='cuda:0')
tensor(14, device='cuda:0') tensor(84, device='cuda:0')
prediction 歐內斯特·拉塞福
tensor(314, device='cuda:0') tensor(315, device='cuda:0')
tensor(14, device='cuda:0') tensor(15, device='cuda:0')
prediction 1927年
tensor(23, device='cuda:0') tensor(24, device='cuda:0')
tensor(66, device='cuda:0') tensor(107, device='cuda:0')
prediction 1992年
tensor(200, device='cuda:0') tensor(201, device='cuda:0')
tensor(29, device='cuda:0') tensor(100, device='cuda:0')
prediction 倫敦
tensor(27, device='cuda:0') tensor(28, device='cuda:0')
tensor(91, device='cuda:0') tensor(91, device='cuda:0')
prediction 1969年
tensor(14, device='cuda:0') tensor(20, device='cuda:0')
tensor(21, device='cuda:0') tensor(45, device='cuda:0')
prediction 位於半屏山之東
tensor(242, device='cuda:0') tensor(246, device='cuda:0')
tensor(35, device='cuda:0') tensor(35, device='cuda:0')
prediction 麥積山石窟
tensor(173, device='cuda:0') tensor(175, device

## Testing

In [ ]:
print("Evaluating Test Set ...")

result = []

model.eval()
with torch.no_grad():
    for data in tqdm(test_loader):
        output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
        result.append(evaluate(data, output))

result_file = "result.csv"
with open(result_file, 'w') as f:	
	  f.write("ID,Answer\n")
	  for i, test_question in enumerate(test_questions):
        # Replace commas in answers with empty strings (since csv is separated by comma)
        # Answers in kaggle are processed in the same way
		    f.write(f"{test_question['id']},{result[i].replace(',','')}\n")

print(f"Completed! Result is in {result_file}")

Evaluating Test Set ...


  0%|          | 0/4957 [00:00<?, ?it/s]

串流輸出內容已截斷至最後 5000 行。
tensor(235, device='cuda:0') tensor(237, device='cuda:0')
prediction 中華民國實業部
tensor(7, device='cuda:0') tensor(12, device='cuda:0')
tensor(62, device='cuda:0') tensor(63, device='cuda:0')
prediction 軍事科研活動
tensor(0, device='cuda:0') tensor(10, device='cuda:0')
tensor(66, device='cuda:0') tensor(71, device='cuda:0')
prediction 中國共產黨紀律檢查機關
tensor(147, device='cuda:0') tensor(148, device='cuda:0')
prediction 大雨
tensor(273, device='cuda:0') tensor(275, device='cuda:0')
tensor(34, device='cuda:0') tensor(36, device='cuda:0')
prediction 民主黨
tensor(169, device='cuda:0') tensor(171, device='cuda:0')
tensor(40, device='cuda:0') tensor(40, device='cuda:0')
prediction 1636年
tensor(49, device='cuda:0') tensor(52, device='cuda:0')
tensor(82, device='cuda:0') tensor(91, device='cuda:0')
tensor(206, device='cuda:0') tensor(208, device='cuda:0')
prediction 人性本善
tensor(348, device='cuda:0') tensor(349, device='cuda:0')
tensor(48, device='cuda:0') tensor(51, device='cuda:0')
predict

In [ ]:
from google.colab import files
files.download('/content/result.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
pred = pd.read_csv('/content/result.csv',index_col='ID')
pred

,Answer
ID,
0,白色羽毛
1,《諸羅縣志》
2,袁晁
3,柏林陸軍學院
4,一局特殊的決勝局
...,...
4952,媒體的輿論
4953,2011年
4954,1960年代
